In [42]:
# imports and loading
import pandas as pd
import numpy as np
import pickle

pickle_in = open('../data/b_to_gene.pkl', 'rb')
b_to_gene = pickle.load(pickle_in)
pickle_in.close()
gene_to_b = {v : k for k, v in b_to_gene.items()}

gene_pres = pd.read_csv('../data/precise_1.0/gene_presence_list.csv', index_col = 0)
iM_table = pd.read_csv('../data/precise_1.0/iM_table.csv', index_col = 0).set_index('name')
iM_to_name = {str(index) : row['name'] for index, row in iM_table.iterrows()}
gene_pres['iModulon'] = [iM_to_name[str(val)] for val in gene_pres['iModulon']]
M = pd.read_csv('../data/precise_1.0/M.csv', index_col = 0).rename(columns = iM_to_name)

# I want a stoichiometric matrix for this, -1 for negative control and 1 for positive control, 0 for nothing
# let's do TRN first
rerun_data_creation = True

f = open('../data/regulonDB_files/NetWorkTFGene.txt', 'r')
lines = f.readlines()
f.close()

# get all genes
genes = set()
for line in lines[38:]:
    genes.add(line.split('\t')[2])
    genes.add(line.split('\t')[4])
TRN_df = pd.DataFrame(index = list(genes), columns = list(genes)).fillna(0)    

# now populate
special_reg_target_to_vals = {}
for line in lines[38:]:
    reg_vals = line.split('\t')[2]
    target = line.split('\t')[4]
    for reg in reg_vals.split(','):
        if line.replace('\n', '').split('\t')[6] not in ['Strong', 'Confirmed']:
            pass#continue I want it all baby
        if line.split('\t')[5] == '+':
            val = 1
        elif line.split('\t')[5] == '-':
            val = -1
        else:
            val = line.split('\t')[5]
        if val != '?' and (TRN_df.loc[reg][target] == 0 or val == TRN_df.loc[reg][target]):
            TRN_df.at[reg, target] = val
        else:
            reg_target = (reg, target)
            if reg_target not in special_reg_target_to_vals:
                special_reg_target_to_vals.update({reg_target : []})
            if TRN_df.loc[reg][target] != 0:
                special_reg_target_to_vals[reg_target].append(TRN_df.loc[reg][target])
            special_reg_target_to_vals[reg_target].append(val)

# let's handle the outliers
unclear_direction = set()
for reg_target, vals in special_reg_target_to_vals.items():
    if '?' in vals: vals.remove('?')
    if len(vals) == 1:
        TRN_df.at[reg_target[0], reg_target[1]] = val
    else:
        if np.mean(vals) == 0:
            unclear_direction.add(reg_target) # not sure what to do with these, for now leave them out

# convert to blattner
checked = set()
old_to_new = {}
for col in TRN_df.columns:
    new = ''
    for val in col.split(','):
        try:
            new += gene_to_b[val]+','
        except:
            if val not in checked:
                print(col)
    new = new[:-1]
    old_to_new.update({col : new})
TRN_df = TRN_df.rename(index = old_to_new, columns = old_to_new)

common_genes = list(set(TRN_df.index).intersection(TRN_df.columns))
TRN_df = TRN_df.loc[common_genes][common_genes]

/home/chris/mambaforge/envs/TRN_model/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/chris/mambaforge/envs/TRN_model/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


sroD


In [45]:
set(TRN_df.values.flatten())

{-1, 0, 1}

In [84]:
TFs = set()
for gene, mini_df in gene_pres.groupby('Gene'):
    if len(mini_df) != 2:
        continue
    # gene is in two iModulons
    # let's make sure one is a repressor and the other a promoter somehow...
    TF_to_reg_dir = {}
    for iM in mini_df.iModulon:
        TF_name = iM_table.loc[iM]['regulator_readable']
        if type(TF_name) == float or ' ' in TF_name:
            continue
        try:
            reg_dir = TRN_df.loc[gene_to_b[TF_name[0].lower()+TF_name[1:]]][gene]
        except:
            continue
        TF_to_reg_dir.update({iM : reg_dir})
    if len(TF_to_reg_dir) == 2 and set(TF_to_reg_dir.values()) == {-1, 1}:
        print(gene)
        print(TF_to_reg_dir)
        print()

b0336
{'PurR-2': -1, 'Nac': 1}

b2208
{'NarL': -1, 'Fnr': 1}

b2239
{'GlpR': -1, 'Crp-2': 1}

b2240
{'GlpR': -1, 'Crp-2': 1}

b3408
{'Fnr': 1, 'Fur-2': -1}

b3409
{'Fnr': 1, 'Fur-2': -1}

b3477
{'Fnr': 1, 'NikR': -1}



In [75]:
TF_to_reg_dir

{}

In [69]:
TRN_df.loc[gene_to_b[TF_name[0].lower()+TF_name[1:]]][gene]

-1

In [35]:
mini_df

,iModulon,Gene
575,ArcA-1,b4705


In [32]:
iM_table

,name,regulator_readable,function,category,n_genes,exp_var,precision,recall,category_num
k,,,,,,,,,
0,YieP,NaN,Unknown,Regulator Discovery,11,0.000128,NaN,NaN,7
1,GlpR,GlpR,Glycerol catabolism,Carbon Source Utilization,9,0.000000,1.000000,1.000000,0
2,YgbI,NaN,Unknown Function,Regulator Discovery,9,0.000000,NaN,NaN,7
3,efeU-repair,NaN,Accounts for repair and expression of efeU operon,Genomic Alterations,8,0.000477,NaN,NaN,9
4,ydcI-KO,NaN,Accounts for ydcI knock-out,Genomic Alterations,3,0.000164,NaN,NaN,9
...,...,...,...,...,...,...,...,...,...
87,FecI,FecI,Ferric citrate transport,Metal Homeostasis,10,0.000000,0.500000,1.000000,3
88,iron-related,NaN,Related to iron metabolism,Biological Enrichment,20,0.000000,NaN,NaN,8
89,RbsR,RbsR,D-ribose catabolism,Carbon Source Utilization,6,0.000637,0.833333,0.555556,0
